In [1]:
import os

from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 함수 작성


리뷰 JSON 파일 로드

In [2]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '깨끗하고 너무 좋았어요', 'stars': 5, 'date': '22시간 전'},
 {'review': '위치도 아주좋고 숙소도 깨끗하고 좋아요~', 'stars': 5, 'date': '2일 전'},
 {'review': '잘쉬고놀다갑니다  감사', 'stars': 5, 'date': '6일 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [3]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(177, 43)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [4]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_END]',
 '[REVIEW_START]근처 볼거리 놀거리가 가까이 있어 편했으나 \n조계사 고층뷰는 의미 없이 배정해 주었으며\n객실 내 청결상태가 별로였음 욕조 머리카락, 수건의 먼지등\n재방문 의사 없음[REVIEW_END]',
 '[REVIEW_START]2월21일(수) 오전7시경, 카운터에 계시던 남자 직원 불친절\n방음 최악[REVIEW_END]']

In [5]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_'

전처리 함수 작성

In [8]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]깨끗하고 너무 좋았어요[REVIEW_END]\n[REVIEW_START]위치도 아주좋고 숙소도 깨끗하고 좋아요~[REVIEW_END]\n[REVIEW_STAR'

### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [9]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

### Baseline 모델 개발

In [10]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [11]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

다양한 리뷰 중에서는 "깨끗하고 위치가 좋아서 만족스러웠다"가 요약될 수 있겠네요.


In [12]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [13]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A의 요약은 매우 간단하고 포괄적입니다. "깨끗하고 위치가 좋아서 만족스러웠다"는 리뷰의 핵심을 짧게 요약했지만, 세부적인 정보나 다양한 리뷰의 내용을 충분히 반영하지 못했습니다.

반면, Assistant B의 요약은 더 구체적이고 상세합니다. 위치, 청결, 직원 서비스, 주변 편의시설, 교통 접근성 등 다양한 측면을 언급하며, 리뷰에서 언급된 여러 요소들을 잘 반영하고 있습니다. 또한, 전반적인 만족도와 재방문 의사까지 포함하여 리뷰의 전반적인 분위기를 잘 전달하고 있습니다.

따라서, Assistant B의 요약이 더 도움이 되고, 리뷰의 내용을 더 정확하고 깊이 있게 반영했다고 판단됩니다.

[[B]]


### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [14]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['다양한 고객들이 나인트리 인사동 숙소를 깨끗하고 위치가 좋다고 평가하며, 서비스와 시설에 대해 긍정적 평가를 했다. 위치가 편리하고 주변에 맛집이 많아 만족하며, 특히 교통과 위치가 최고의 장점으로 꼽는다. 가격 대비 시설이 좋고, 편한 휴식을 취할 수 있는 좋은 숙박을 경험했다. Car park charges are a downside but overall, the stay was comfortable and satisfactory.',
 '다양한 리뷰에서 숙소가 깨끗하고 위치가 좋다는 호평을 받고, 일부 방음이 조금 아쉽다는 언급도 있었습니다. 종종 이용하는 손님들도 있고, 가성비가 좋아서 재방문하고 싶다는 의견도 있었습니다.친절한 직원과 청결한 시설에 대한 긍정적인 평가가 많이 있습니다.',
 '다양한 리뷰 속에서 호텔의 위치, 깨끗함, 서비스, 시설 등에 대한 긍정적인 평가가 많았어요. 특히 인사동과의 접근성이 좋다는 평가가 많았습니다. 앞으로도 자주 이용하고 싶은 호텔로 손꼽히는 곳이었어요.',
 '다양한 리뷰 중에서 숙소 위치와 청결도에 대한 긍정적 평가가 많이 나왔습니다. 고객들은 대체로 만족하며 깔끔하고 좋은 서비스를 경험했다는 의견을 남겼습니다. 숙소 위치가 인사동이나 주변 관광지에 인근하여 관광하기에 편리하다는 점도 많이 언급되었습니다. 서비스나 시설 등 모든 면에서 만족스러운 경험을 한 고객들이 많았습니다.',
 '다양한 리뷰들 중에는 "깨끗하고 위치 좋음" "시설 만족도 높고 관광지 인근" "인사동 접근성 좋고 교통도 편리" "친절한 직원들과 편안한 휴식" "가격 대비 만족도 높음" 등이 있다.',
 '다양한 리뷰 중 "깨끗하고 위치가 좋다"는 내용이 주로 나타나는 것 같습니다. 위치와 청결도에 대한 언급이 많이 보여집니다.',
 '다양한 리뷰가 있지만 전반적으로 깨끗한 시설과 좋은 위치에 대한 만족이 높은 숙소로 요약할 수 있겠어요.',
 '다른 이용객들이 숙소를 깨끗하고 위치가 좋아서 만족하며 쉬고 놀았고, 특히 주변 관광지와 교통

In [15]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')

    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:54<00:00,  5.49s/it]

Wins: 0, Losses: 10, Ties: 0


### 모델 고도화 1 - 조건들 명시

In [17]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:52<00:00,  5.26s/it]

Wins: 2, Losses: 8, Ties: 0


### 모델 고도화 2 - 입력 데이터의 품질 증가

In [30]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [23]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:49<00:00,  4.99s/it]

Wins: 7, Losses: 3, Ties: 0


### 모델 고도화 3 - Few-Shot Prompting

In [24]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:59<00:00,  5.93s/it]

Wins: 4, Losses: 6, Ties: 0


In [25]:
summaries

['인사동과 조계사를 도보로 품은 최고의 위치에 위치한 숙소입니다. 주변 관광지와 교통도 편리하며 시설과 서비스 모두 최고라는 호평을 받고 있습니다. 깨끗하고 편안한 만족스러운 숙박 경험을 제공받았습니다. 나인트리 인사동은 다양한 맛집과 편의시설이 가까이에 있어 편리한 여행을 즐길 수 있는 곳으로 추천합니다. 위치와 서비스 모두 최고로 평가받고 있으며 다음 서울 여행 때도 묵고 싶은 숙소로 손꼽힙니다.',
 '고품격 숙소로, 주변 관광지를 품은 최고의 입지를 자랑합니다. 3번째 방문자로서 좋은 평가를 받았고, 시설과 서비스로 최고의 숙소라는 평가를 받았습니다. 위치와 교통의 편리함을 강조한 후에, 다음 방문을 고려하고 있는 다양한 사용자 리뷰들이 있습니다. 위치와 가격, 시설로 인해 만족도가 높은 숙박 경험이 공유되고 있습니다.',
 '고객님의 소중한 후기를 요약해드리겠습니다. \n숙소는 말할 것도 없이 주변에 있는 인사동과 조계사 위로 경복궁까지 도보로 품은 입지가 최고의 장점입니다. \n모든 편의시설이 완벽하게 갖춰져 있어서 편안하고 편리한 숙박을 경험하실 수 있습니다. 이번에도 편안하게 휴식을 취하고 오셨다는 후기들이 많아요^^',
 '인사동과 조계사를 도보로 품은 최적의 입지에 위치한 숙소입니다. 관광지와 교통이 편리하여 재방문하고 싶은 인기 숙박지입니다. 시설과 서비스 모두 뛰어나며, 깨끗하고 편안한 휴식을 제공받을 수 있습니다. 위치와 교통편이 좋아 즐거운 여행을 즐기기에 최적의 선택입니다. 호캉스나 가족 여행에 추천드리는 숙소입니다.',
 '고퀄리티의 숙소인 나인트리인사동은 조계사와 경복궁을 도보로 즐길 수 있는 최고의 입지를 자랑합니다. 위치와 교통이 훌륭한 이 숙소는 시설과 서비스가 빠짐없이 완벽하여, 편안한 휴식을 제공합니다. 또한, 인사동의 관광지와 가까운 위치로 여행객들에게 강력한 선택지로 손꼽힙니다. 맛집이나 볼거리를 찾으러다니기에도 최적의 환경을 제공합니다. 최고의 위치와 서비스로 과거 방문한 고객들의 기대를 충족시켜, 만족도를 높이는

In [26]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:50<00:00,  5.10s/it]

Wins: 9, Losses: 1, Ties: 0


In [27]:
summaries

['이 숙소는 인사동에 위치하여 주변 관광지와의 접근성이 매우 좋으며, 교통편 또한 편리하게 이용할 수 있습니다. 방과 시설 모두 깔끔하고 청결하며, 직원들의 친절한 서비스도 만족스러운 것으로 평가받고 있습니다. 특히 위치가 최고라고 극찬하는 리뷰도 많이 있습니다. 다만 일부 고객들은 방음이나 주차요금 등에 대한 아쉬움을 표현하기도 했습니다. 이 숙소는 관광객이나 가족 여행객들에게 매력적인 숙박지로 평가됩니다.',
 '요약 결과:\n나인트리 인사동 호텔은 인사동 한복판에 위치하여 관광지까지 접근성이 뛰어나며, 시설과 서비스 모두 훌륭한 호텔로 평가받고 있습니다. 깨끗하고 친절한 직원들이 인상적이며, 청결도와 편의성을 높게 평가받고 있습니다. 또한 위치적으로도 매우 우수하며, 근처에 맛집과 명소가 많아 먹거리나 즐길 거리가 풍부합니다. 다만, 일부 리뷰에서 방음이나 주차요금 등에 대한 약간의 불만이 있었지만, 전반적으로 고객들의 만족도가 높은 호텔로써, 재방문 의사가 매우 높은 편입니다. 결과적으로 가성비로 높은 호텔로 평가받고 있습니다.',
 '요약:\n나인트리 인사동은 숙소 자체의 품질이나 서비스, 위치 등에 대해 매우 높은 만족도를 보여주고 있습니다. 주변 인사동과 명동 등 관광지까지의 접근성도 좋고, 근처에 맛집과 편의시설이 많아 편리한 여행을 지낼 수 있습니다. 객실 청결도도 훌륭하며, 일부 소음이 있을 수는 있지만 전반적으로 가성비 좋은 호텔로 호텔 내시설, 위치, 서비스 등 모든면에서 만족을 느낀 고객들이 많은 편입니다.',
 '이 숙소는 대부분의 리뷰에서 위치가 좋고 깨끗하며 인사동과 관광지에 가까워서 편리하다는 평가를 받았습니다. 직원들의 친절함과 객실 청결도도 높게 평가받았으며, 가성비가 좋은 숙소로 확인되었습니다. 일부 방음이 아쉬운 점으로 지적되었지만 전반적으로 고객들의 만족도가 높았습니다.위치가 좋고 인사동과 가까운 숙소라는 점이 많은 이용자들에게 만족을 준 것으로 나타났습니다.',
 '이 숙소는 위치가 매우 훌륭하며, 교통편도 좋고 주변

In [31]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

In [32]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:47<00:00,  4.77s/it]

Wins: 9, Losses: 1, Ties: 0


In [33]:
summaries

['이 호텔은 주변 관광지와 교통이 좋고 청결한 시설을 자랑합니다. 게스트는 위치의 편리성과 깨끗한 객실을 칭찬했으며, 직원의 친절함과 뷰에 대한 언급도 많이 나왔습니다. 다만 일부 리뷰에서는 방음에 대한 언급이 있지만 전체적으로 고객들은 만족하며 재방문하고 싶다는 의사를 밝혔습니다.격한 소리가 들린다는 의견이 있습니다. 이야기하는 소리가 들렸다.[REVIEW_END]',
 '이 숙소는 위치가 인사동 중심에 있어서 관광하기에 편리하며, 청결하고 깔끔한 시설을 자랑합니다. 많은 이용객들이 직원들의 친절함에 대해 언급하며, 교통 접근성도 좋다는 평을 받았습니다. 또한, 주변 시설과 맛집들이 풍부하여 편의성이 뛰어나다고 말했고, 다음에 또 방문하고 싶다는 의견이 많이 나타났습니다. 전반적으로 고객들은 이 숙소에서의 청결함, 위치, 서비스에 만족하며 재방문 의사가 높은 것으로 나타났습니다.',
 '요약 결과:\n이 숙소는 높은 청결도, 편리한 위치, 그리고 친절한 직원들이 장점으로 꼽힙니다. 주변 관광지에 접근성이 좋고, 객실은 깔끔하며 시설도 모던합니다. 일부 방음이 아쉬운 점을 제외하고 객실 상태와 편의시설, 서비스 등에 매우 만족한 고객들이 많이 있습니다. 또한, 가격 대비 가성비가 높다는 평가도 받고 있습니다. 앞으로도 재방문하고 싶은 숙소로 손꼽히고 있습니다.',
 '이 숙소는 인사동과 주변 관광지에 위치하여 접근성이 뛰어나며, 깨끗한 시설과 친절한 서비스를 자랑합니다. 객실은 깔끔하고 편리한 시설을 갖추고 있어 다양한 이용객들로부터 좋은 후기를 받고 있습니다. 주변에 맛집이나 쇼핑 시설이 가까이에 있어서 이용자들의 만족도가 높고, 위치가 좋아 좋은 추억을 만들기에 적합한 숙소로 평가받고 있습니다. 방음이 좀 아쉬움이 있을 수 있지만, 전반적으로 고객들은 재방문 의사가 높은 호텔로서 호감을 표현하고 있습니다.',
 '이 숙소는 인사동과 조계사를 걸어서 즐길 수 있는 입지에 위치하여 접근성이 좋고, 교통편도 편리하다는 리뷰들이 많이 있습니다. 다양한 시설과 서